Nous souhaitons étudier le comportement des trajets des taxis new yorkais. Pour cela nous
allons calculer les indicateurs ci-dessous :
● la vitesse moyenne de chaque trajet,
● le nombre de trajets effectués en fonction du jour de la semaine,
● le nombre de trajets effectués en fonction de l’horaire de la journée par tranche de 4h,
● le nombre de km parcourus par jour de la semaine.

In [2]:
# Install dependencies
!pip install pyspark

In [5]:
# Get Spark Context
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import StructType,StructField, StringType, IntegerType , BooleanType
spark = SparkSession.builder.appName('pyspark - NYC Taxi Trip Duration').getOrCreate()
sc = spark.sparkContext

In [2]:
print(sc)

<SparkContext master=local[*] appName=pyspark - NYC Taxi Trip Duration>


In [3]:
df = spark.read.csv("data/train.csv")
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)



In [7]:
df.show(5,False)

+---------+---------+-------------------+-------------------+---------------+-------------------+------------------+-------------------+------------------+------------------+-------------+
|_c0      |_c1      |_c2                |_c3                |_c4            |_c5                |_c6               |_c7                |_c8               |_c9               |_c10         |
+---------+---------+-------------------+-------------------+---------------+-------------------+------------------+-------------------+------------------+------------------+-------------+
|id       |vendor_id|pickup_datetime    |dropoff_datetime   |passenger_count|pickup_longitude   |pickup_latitude   |dropoff_longitude  |dropoff_latitude  |store_and_fwd_flag|trip_duration|
|id2875421|2        |2016-03-14 17:24:55|2016-03-14 17:32:30|1              |-73.982154846191406|40.767936706542969|-73.964630126953125|40.765602111816406|N                 |455          |
|id2377394|1        |2016-06-12 00:43:35|2016-06-12 00: